<a href="https://colab.research.google.com/github/NILEG/TensorFlow/blob/main/005(Image_Generator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

####Loading Dataset

In [2]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2023-09-02 13:26:15--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   211MB/s    in 0.7s    

2023-09-02 13:26:16 (211 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [4]:
#Creating Data Loader pipeline using Data Generator
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
train_data=train_generator.flow_from_directory("./horse-or-human/", target_size=(300,300), batch_size=128, class_mode="binary")

Found 1027 images belonging to 2 classes.


In [5]:
#Total images of each class
import os
horses=len(os.listdir('./horse-or-human/horses'))
humans=len(os.listdir('./horse-or-human/humans'))
print("Horse Images: "+ str(horses)+ "\nHuman Images: "+str(humans))

Horse Images: 500
Human Images: 527


####Creating the model

In [10]:
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(300,300,3)),
    #First Layer
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(strides=(2,2)),
    #Second Layer
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(strides=(2,2)),
    #Third Layer
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(strides=(2,2)),
    #Forth Layer
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(strides=(2,2)),
    #Fifth Layer
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(strides=(2,2)),
    #Flatten Layer
    tf.keras.layers.Flatten(),
    #Dense Layer
    tf.keras.layers.Dense(units=512, activation="relu"),
    #Output Layer
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(train_data, steps_per_epoch=8, epochs=15, verbose=1) #step_per_epoch=floor(total_length_of_images/batch_size) ==> 1027/128 = 8

Epoch 1/15
8/8 [==============================] - 9s 765ms/step - loss: 0.7699 - accuracy: 0.4861
Epoch 2/15
8/8 [==============================] - 7s 830ms/step - loss: 0.6867 - accuracy: 0.6652
Epoch 3/15
8/8 [==============================] - 8s 975ms/step - loss: 0.6298 - accuracy: 0.7031
Epoch 4/15
8/8 [==============================] - 8s 936ms/step - loss: 0.5566 - accuracy: 0.7430
Epoch 5/15
8/8 [==============================] - 7s 889ms/step - loss: 0.3923 - accuracy: 0.8443
Epoch 6/15
8/8 [==============================] - 7s 827ms/step - loss: 0.2884 - accuracy: 0.8643
Epoch 7/15
8/8 [==============================] - 8s 911ms/step - loss: 0.2065 - accuracy: 0.9321
Epoch 8/15
8/8 [==============================] - 8s 929ms/step - loss: 0.2654 - accuracy: 0.9066
Epoch 9/15
8/8 [==============================] - 7s 970ms/step - loss: 0.7614 - accuracy: 0.8543
Epoch 10/15
8/8 [==============================] - 7s 843ms/step - loss: 0.0897 - accuracy: 0.9711
Epoch 11/15
8/8 [==

####Testing with own data(images from pexels):

In [13]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = tf.keras.utils.load_img(path, target_size=(300, 300))
  x = tf.keras.utils.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)
  classes = model.predict(x, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving worried-girl-413690_640.jpg to worried-girl-413690_640 (1).jpg
Saving white-horse-3010129_1280.jpg to white-horse-3010129_1280 (2).jpg
1/1 [==============================] - 0s 26ms/step
[0.99879897]
worried-girl-413690_640 (1).jpg is a human
1/1 [==============================] - 0s 23ms/step
[0.01769603]
white-horse-3010129_1280 (2).jpg is a horse
